# The FD Chat Bot Format

In this notebook a sincere try to create chat bot/conversational inerface to book a Fixed Deposit for a set of NBFC. Data used in this Notebook is only Sample and Names of NBFC are also sample, these do not reflect any real data

## Setup

In [104]:
import os
import openai
import panel as pn  # GUI
import pandas as pd
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

Methods to connect the Open AI APIs and process the messages

In [105]:
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

# Reading the Rates Data to use in FD Bot
We are trying to automate the collection of user prompts and assistant responses to build a  FD Booking Bot.and these rates and FD booking options will be used in that

In [106]:
rates = pd.read_csv('FDRates.csv')
rates_dict = rates.to_dict(orient='records')

Checking if the data has been loaded correctly ?

In [ ]:
#rates.head(5)
#rates_dict

In [107]:
prompt_text=f"""
You are a OrderBot, an automated service to collect orders for a Fixed Deposit (FD). \
use the below steps to process an order. \
1. You first greet the customer, then collects the FD booking, \
2. You wait to collect the entire information such Customer details ( Name, DOB and PAN ),Amount and duration in months, Frequency \
3. and You asks it's for which provider ? - Provider1 or Provider2. \
4. then summarize it after finding the rates for the above mentioned provider from The Fixed Deposit Rates section within ```.\ 
and then check for a final time if the customer wants to add anything else to book a Fixed Deposit \
5. Finally you collect the payment.\
6. Make sure to clarify all options, extras and TnC to book a Fixed Deposit \
7. You respond in a short, very conversational friendly style. \
8. The Fixed Deposit options and intesrest rates are provided below in a dictioray format. \
```{rates_dict}```
"""

In [ ]:
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':prompt_text} ]  # accumulate messages
inp = pn.widgets.TextInput(value="Hi", width=800, placeholder='Enter the FD inputs here…',disabled=False)


In [109]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    # Append message panels
    user_pane = pn.Row('User:', pn.pane.Markdown(prompt, width=800))
    assistant_pane = pn.Row('Assistant:', pn.pane.Markdown(response, width=800, height_policy='auto', styles={'background-color': '#F6F6F6'}))
    
    panels.extend([user_pane, assistant_pane])
 
    return pn.Column(*panels)

def on_submit(event):
    # Check if the event corresponds to a key press
    if event.type == 'changed':
        # Check if Enter key is pressed
        if event.new.endswith('\n'):
            # Call collect_messages function
            collect_messages(inp)


inp.param.watch(on_submit, 'value')


Watcher(inst=TextInput(placeholder='Enter the FD i..., value='Hi', width=800), cls=<class 'panel.widgets.input.TextInput'>, fn=<function on_submit at 0x0000027836845A20>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

Please enter your text in CAPS in input box and press the Chat Button, some issue with small case ( will try to fix it in next version) ot if you are able to fix it please share the fix.

In [110]:

button_conversation = pn.widgets.Button(name="Process the FD Booking", button_type='primary')
interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height_policy='auto'),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'da570b55-1f43-4cea-b585-0296d517cb9f': {'version…

# Verfication and Next Steps
By executing the below code, you can see bot has collected the information correctlty even of the order of the information was random
Json output provided by below method can be passed to the FD Booking Service which can process rhe request
FD Booking Service - Not part of this code, has to be developed seprately

In [111]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous Fixed Deposit booking.\
 The fields should be 1) Name DOB PAN 2) Amount 3) duration 4) rate  5) FD Provider 5) Frequency Type'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
    "Name": "Sanjay Gupta",
    "DOB": "5/6/1989",
    "PAN": "ABCDEFGH",
    "Amount": "₹9,000",
    "Duration": "24 months",
    "Interest Rate": "7.81%",
    "FD Provider": "Provider1",
    "Frequency Type": "Monthly"
}
